## Trading a Delta Hedged Straddle

1.  Buy 10 contracts of puts and calls at the strike closest to current stock price every morning
    a. Expiry that is closest to today but at least 5 trading days out.
    b. Hedge remaining delta
2.  Re-hedge if residual delta > 2 contracts
2.  Exit the trade at EOD.
5.  We assume no slippage (i.e. entry and exit at mid price) and commission of 1/2 a cent for stock trades and 1 dollar for option trades

First lets generate some option prices based on the underlying stock price.  We will generate some random volatility numbers, and use Black Scholes to generate prices.  We will add all this data to a strategy context that we can later use from our strategy and avoid excessive global variables.

In [5]:
%%checkall
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Sequence
import pyqstrat as pq
from types import SimpleNamespace


_logger = pq.get_child_logger(__name__)


def get_symbol(put_call: str, strike: int, expiry: np.datetime64) -> str:
    '''Create option symbol from parameters'''
    return f'{put_call}-{strike}-{expiry}'


def parse_symbol(symbol: str) -> tuple[str, int, np.datetime64]:
    '''Break down option symbol into put_call, strike, expiry'''
    split = symbol.split('-', maxsplit=2)
    return (split[0], int(split[1]), np.datetime64(split[2]))
       

@dataclass
class StraddleEntryRule:
    expiries: dict[np.datetime64, np.ndarray]
    strikes: dict[tuple[np.datetime64, str, np.datetime64], np.ndarray]
    umids: dict[np.datetime64, float]
        
    def __call__(self,
                 contract_group: pq.ContractGroup,
                 i: int,
                 timestamps: np.ndarray,
                 indicator_values: pq.SimpleNamespace,
                 signal_values: pq.SimpleNamespace,
                 account: pq.Account,
                 current_orders: Sequence[pq.Order],
                 strategy_context: pq.StrategyContextType) -> list[pq.Order]:
        
#         for order in current_orders:
#             if order.contract.is_basket(): 
#                 order.cancel()
                
        timestamp = timestamps[i]
        date = timestamp.astype('M8[D]')
        expiries = self.expiries[date]
        expiry = expiries[pq.np_find_closest(expiries, date + np.timedelta64(30, 'D'))]
        put_strikes = self.strikes[(date, 'P', expiry)]
        call_strikes = self.strikes[(date, 'C', expiry)]
        umid = self.umids[timestamp]
        cidx = pq.np_find_closest(call_strikes, umid)
        pidx = pq.np_find_closest(put_strikes, umid)
        found = False
        for j in range(10):
            put_strike = put_strikes[pidx]
            call_strike = call_strikes[cidx]
            # _logger.info(f'trying strike: {put_strike} {call_strike} index: {pidx} {cidx} i: {i} timestamp: {timestamp}')
            put_symbol = get_symbol('P', put_strike, expiry)
            call_symbol = get_symbol('C', call_strike, expiry)
            call_contract = contract_group.get_contract(call_symbol)
            put_contract = contract_group.get_contract(put_symbol)
            if call_contract is None: call_contract = pq.Contract.create(call_symbol, contract_group, expiry, 100)
            if put_contract is None: put_contract = pq.Contract.create(put_symbol, contract_group, expiry, 100)
            put_delta = context.get_delta(put_contract, timestamps, i, context)
            call_delta = context.get_delta(call_contract, timestamps, i, context)
            if np.isfinite(put_delta) and np.isfinite(call_delta): 
                # _logger.info(f'found deltas: {put_delta} {call_delta}')
                found = True
                break
            cidx += 1
            pidx -= 1
        if not found: return []
        symbol = f'{put_symbol}_{call_symbol}'
        contract = contract_group.get_contract(symbol)
        if contract is None: 
            contract = pq.Contract.create(symbol, 
                                          contract_group,
                                          multiplier=100, 
                                          components=[(put_contract, 1), (call_contract, 1)])
        order = pq.MarketOrder(contract=contract, timestamp=timestamp, qty=10, reason_code='ENTER_STRADDLE')
        _logger.info(f'ORDER: {timestamp} {order}')
        # import pdb; pdb.set_trace()
        return [order]
    
    
@dataclass
class RehedgeRule:
    '''
    A rule to re-hedge deltas on a straddle
    Args:
        reason_code: the reason for the orders (used for display purposes)
        price_func: the function this rule uses to get market prices
    '''
    reason_code: str
    price_func: pq.PriceFunctionType
        
    def __init__(self, 
                 reason_code: str, 
                 price_func: pq.PriceFunctionType) -> None:
        self.reason_code = reason_code
        self.price_func = price_func
        
    def __call__(self,
                 contract_group: pq.ContractGroup,
                 i: int,
                 timestamps: np.ndarray,
                 indicator_values: SimpleNamespace,
                 signal_values: SimpleNamespace,
                 account: pq.Account,
                 current_orders: Sequence[pq.Order],
                 strategy_context: pq.StrategyContextType) -> list[pq.Order]:
        # import pdb; pdb.set_trace()
        timestamp = timestamps[i]
        positions = account.positions(context.opt_contract_group, timestamp)
        
        orders: list[pq.Order] = []
        for (contract, qty) in positions:
            if not contract.is_basket(): continue
            put_contract, put_ratio = contract.components[0]
            call_contract, call_ratio = contract.components[1]
            hedge_contract, target_hedge_qty = get_hedge(
                put_contract, call_contract, qty * put_ratio, qty * call_ratio, timestamps, i, context)
            if np.isnan(target_hedge_qty): return []  # can have nan deltas sometimes
            hedge_positions = account.positions(contract_group, timestamp)
            pq.assert_(len(hedge_positions) in [0, 1], f'unexpected num of hedge positions: {hedge_positions}')
            curr_hedge_qty = 0
            if len(hedge_positions) == 1:
                curr_hedge_qty = hedge_positions[0][1]
            hedge_qty = target_hedge_qty - curr_hedge_qty
            if hedge_qty == 0: return []
            hedge_order = pq.MarketOrder(contract=hedge_contract, 
                                         timestamp=timestamp, 
                                         qty=int(hedge_qty), 
                                         reason_code='REHEDGE')
            orders.append(hedge_order)
            _logger.info(f'ORDER: {timestamp} {hedge_order}')
        return orders
    

def get_hedge(put: pq.Contract, 
              call: pq.Contract, 
              put_qty: int, 
              call_qty: int, 
              timestamps: np.ndarray, 
              i: int, 
              context: pq.StrategyContextType) -> tuple[pq.Contract, int]:
    # import pdb; pdb.set_trace()
    delta: float = 0
    delta += context.get_delta(put, timestamps, i, context) * put_qty
    delta += context.get_delta(call, timestamps, i, context) * call_qty
    hedge_contract = context.spx_contract
    pq.assert_(hedge_contract is not None)
    
    hedge_qty = np.round(-100 * delta)
    return hedge_contract, hedge_qty


def get_expiries(prices: pd.DataFrame) -> dict[np.datetime64, np.ndarray]:
    _expiries = prices.groupby(['date']).expiry.unique()
    expiries = {_expiries.index.values[i].astype('M8[D]'): 
                np.sort(_expiries.values[i].astype('M8[D]')) for i in range(len(_expiries))}
    return expiries


def get_strikes(prices: pd.DataFrame) -> dict[tuple[np.datetime64, str, np.datetime64], np.ndarray]:
    _strikes = prices.groupby(['date', 'put_call', 'expiry']).strike.unique()
    for i in range(len(_strikes)):
        _strikes[i] = _strikes[i][_strikes[i] % 100 == 0]
    strikes = {(_strikes.index.get_level_values(0).values[i].astype('M8[D]'), 
                _strikes.index.get_level_values(1)[i], 
                _strikes.index.get_level_values(2).values[i].astype('M8[D]')): np.sort(_strikes.values[i]) 
               for i in range(len(_strikes))}
    return strikes


def get_price_function(prices: pd.DataFrame, field_name: str) -> pq.PriceFunctionType:
    price_dict: dict[str, tuple[np.ndarray, np.ndarray]] = {}
    for symbol in np.unique(prices.symbol.values):
        sym_prc = prices[['timestamp', field_name]][prices.symbol == symbol].sort_values(by='timestamp')
        _timestamps = sym_prc.timestamp.values.astype('M8[m]')
        _prices = sym_prc[field_name].values
        price_dict[symbol] = (_timestamps, _prices)
    spx_prices = prices[['timestamp', 'umid']].sort_values(by=['timestamp']).drop_duplicates(subset=['timestamp'])
    price_dict['SPX'] = (spx_prices.timestamp.values.astype('M8[m]'), spx_prices.umid.values)
    return pq.PriceFuncArrayDict(price_dict=price_dict)


@dataclass
class BasketOrderMarketSimulator:
    '''
    A function object with a signature of MarketSimulatorType.
    It can take into account slippage and commission
    >>> pq.ContractGroup.clear()
    >>> pq.Contract.clear()
    >>> cg = pq.ContractGroup.create('test_cg')
    >>> put_symbol, call_symbol = 'SPX-P-3500-2023-01-19', 'SPX-C-4000-2023-01-19'
    >>> put_contract = pq.Contract.create(put_symbol, cg)
    >>> call_contract = pq.Contract.create(call_symbol, cg)
    >>> basket = pq.Contract.create('test_contract', cg)
    >>> basket.components = [(put_contract, -1), (call_contract, 1)]
    >>> timestamp = np.datetime64('2023-01-03 14:35')
    >>> price_func = pq.PriceFuncDict({put_symbol: {timestamp: 4.8}, call_symbol: {timestamp: 3.5}})
    >>> order = pq.MarketOrder(contract=basket, timestamp=timestamp, qty=10, reason_code='TEST')
    >>> sim = BasketOrderMarketSimulator(price_func=price_func, slippage_per_trade=0)
    >>> out = sim([order], 0, np.array([timestamp]), {}, {}, SimpleNamespace())
    >>> assert(len(out) == 1)
    >>> assert(math.isclose(out[0].price, -1.3))
    >>> assert(out[0].qty == 10)
    '''
    slippage: float
    price_func: pq.PriceFunctionType
        
    def __init__(self,
                 price_func: pq.PriceFunctionType,
                 slippage_per_trade: float = 0.) -> None:
        '''
        Args:
            price_func: A function that we use to get the price to execute at
            slippage_per_trade: Slippage in local currency. Meant to simulate the difference
            between bid/ask mid and execution price 
        '''
        self.price_func = price_func
        self.slippage = slippage_per_trade
    
    def __call__(self,
                 orders: Sequence[pq.Order],
                 i: int, 
                 timestamps: np.ndarray, 
                 indicators: dict[pq.ContractGroup, SimpleNamespace],
                 signals: dict[pq.ContractGroup, SimpleNamespace],
                 strategy_context: SimpleNamespace) -> list[pq.Trade]:
        trades = []
        timestamp = timestamps[i]
        for order in orders:
            contract = order.contract
            if not isinstance(order, pq.MarketOrder) and not isinstance(order, pq.LimitOrder): continue
            prices_found = True
            raw_price = 0.
            for (_contract, ratio) in contract.components:
                raw_price += self.price_func(_contract, timestamps, i, strategy_context) * ratio
                if np.isnan(raw_price):
                    prices_found = False
                    break
            if not prices_found: continue
            slippage = self.slippage
            if order.qty < 0: slippage = -slippage
            price = raw_price + slippage
            if isinstance(order, pq.LimitOrder):
                if np.isfinite(order.limit_price):
                    if ((abs(order.qty > 0) and order.limit_price > price) 
                            or (abs(order.qty < 0) and order.limit_price < price)):
                        _logger.debug(f'limit_price: {order.limit_price} not met price: {price}')
                        continue
            # market order
            trade = pq.Trade(order.contract, order, timestamp, order.qty, price)
            _logger.info(f'Trade: {timestamp.astype("M8[m]")} {trade}')
            trades.append(trade)
            order.fill()
        return trades


if __name__ == '__main__':
    pq.set_defaults()
    pq.Contract.clear()
    pq.ContractGroup.clear()

    prices = pd.read_csv('./support/spx_options.csv.gz', parse_dates=['timestamp'])
    prices = prices[['timestamp', 'symbol', 'umid', 'c', 'delta']]
    
    prices['date'] = prices.timestamp.values.astype('M8[D]')
    # prices = prices[prices.date == "2023-01-03"]

    # remove prices outside regular trading hours (9:30 am and 4 pm)
    minute = (prices.timestamp - prices.date) / np.timedelta64(1, 'm')
    prices = prices[(minute > 9 * 60 + 30) & (minute < 16 * 60)]

    splits = prices.symbol.str.split('-', n=2, expand=True)
    prices['put_call'] = splits[0]
    prices['strike'] = splits[1].astype(int)
    prices['expiry'] = splits[2].astype(np.datetime64)

    data = prices[['timestamp', 'umid']].sort_values(by=['timestamp']).drop_duplicates(subset=['timestamp'])
    data['date'] = data.timestamp.values.astype('M8[D]')
    data['hour'] = data.timestamp.dt.hour
    
    # Beginning and end of day and rehedge signals
    # Try 6 five-minute periods in case we don't get valid prices (every strike is not traded in every bar)
    bod = pd.Series(np.where(data.date != data.date.shift(1), 1, np.nan))
    bod = bod.fillna(method='ffill', limit=6)  
    data['bod'] = np.where(bod == 1, True, False)
    
    eod = pd.Series(np.where(data.date != data.date.shift(-1), 1, np.nan))
    # Try getting out 30 minutes before close so we have 6 bars to try and get out
    eod = eod.fillna(method='bfill', limit=6)  
    data['eod'] = np.where(eod == 1, True, False)
    
    data['rehedge'] = (data.hour != data.hour.shift(1))
    
    strat_builder = pq.StrategyBuilder(data)
    context = pq.StrategyContextType()
    price_func = get_price_function(prices, 'c')
    delta_func = get_price_function(prices, 'delta')
    context.get_price = price_func
    context.get_delta = delta_func
    strat_builder.set_strategy_context(context)
    strat_builder.set_price_function(price_func)
    strat_builder.add_market_sim(BasketOrderMarketSimulator(price_func, 0.))
    
    expiries = get_expiries(prices)
    strikes = get_strikes(prices)
    umids = {data.timestamp.values[i].astype('M8[m]'): data.umid.values[i] for i in range(len(data))}

    opt_cg = pq.ContractGroup.create('OPTIONS')
    strat_builder.add_contract_group(opt_cg)
    context.opt_contract_group = opt_cg

    hedge_cg = pq.ContractGroup.create('HEDGES')
    spx = pq.Contract.create('SPX', contract_group=hedge_cg)
    strat_builder.add_contract_group(hedge_cg)
    context.spx_contract = spx

    # add rules to the strategy
    straddle_entry_rule = StraddleEntryRule(expiries, strikes, umids)
    strat_builder.add_series_rule('bod', straddle_entry_rule, position_filter='zero', contract_groups=[opt_cg])
    rehedge_rule = RehedgeRule('REHEDGE', context.get_price)
    strat_builder.add_series_rule('rehedge', rehedge_rule, contract_groups=[hedge_cg])
    close_rule = pq.ClosePositionExitRule('EOD', context.get_price)
    strat_builder.add_series_rule('eod', close_rule, position_filter='nonzero')

    # build and run strategy
    strategy = strat_builder()
    strategy.run()

running typecheck
<string>:123: error: Name "math" is not defined  [name-defined]
<string>:123: error: Name "target_hedge_quantity" is not defined  [name-defined]
Found 2 errors in 1 file (checked 1 source file)

running flake8
stdin:40:1: E115 expected an indented block (comment)
stdin:41:1: E115 expected an indented block (comment)
stdin:42:1: E115 expected an indented block (comment)
stdin:123:16: F821 undefined name 'math'
stdin:123:27: F821 undefined name 'target_hedge_quantity'

[2023-10-21 18:48:56.508 __call__] ORDER: 2023-01-03T09:35 P-3900-2023-01-20_C-3900-2023-01-20 2023-01-03 09:35:00 qty: 10 ENTER_STRADDLE OrderStatus.OPEN
[2023-10-21 18:48:56.509 __call__] Trade: 2023-01-03T09:40 P-3900-2023-01-20_C-3900-2023-01-20 2023-01-03 09:40:00 qty: 10 prc: 143.65 order: P-3900-2023-01-20_C-3900-2023-01-20 2023-01-03 09:35:00 qty: 10 ENTER_STRADDLE OrderStatus.OPEN


NameError: Exception: name 'math' is not defined at rule: <class '__main__.RehedgeRule'> contract_group: HEDGES index: 5

In [3]:
strategy.df_roundtrip_trades()

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,P-3900-2023-01-20_C-3900-2023-01-20,100,2023-01-03 09:40:00,NaT,10,143.65,NaN,ENTER_STRADDLE,None,0,NaN,0
1,SPX,1,2023-01-03 10:05:00,NaT,228,0,NaN,REHEDGE,None,0,NaN,0


In [4]:
np.round(np.nan)

nan